In [ ]:

# Предположение: ожидаем увидеть конверсию, которая не позволит отклонить нулевую
# гипотезу о случайной разнице конверсий двух веб-страниц
# Нулевая гипотеза: разница конверсий (количества кликов в нашем случае) случайна
# Альтернативная гипотеза: разница конверсий не случайна
from ipywidgets import interact,IntSlider


from IPython.display import display
from statsmodels.stats.proportion import proportions_ztest, proportion_confint


import pandas as pd
from math import ceil

pd.plotting.register_matplotlib_converters()
from scipy import stats
from scipy.stats import mannwhitneyu
import statsmodels.api as sm
import statsmodels.stats.api as sms
import warnings
from itertools import product
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from io import StringIO
from tqdm.auto import tqdm
from scipy.stats import norm

%load_ext autoreload
%autoreload 2

import numpy as np
from matplotlib import pyplot as plt

import sys
sys.path.append('..')

from stattests.tests import *
from stattests.generation import generate_data
from stattests.utils import plot_cdf, plot_summary, codenames2titles,  frame_from_params, plot_from_params, save_gif_and_show
from stattests.data import rpv, apply_all_tests
from tqdm import tqdm

import seaborn as sns
sns.set_style("whitegrid")

from scipy.stats import arcsine
from scipy.stats import beta



colors = sns.color_palette("deep")



from typing import Tuple, Dict, Optional, Set, List

import imageio
from IPython.core.display import HTML
from matplotlib.axes import Axes




class Export_Data:

#     filepath = 'C:/Users/Dell/Projects/mind_set/ab_testing/ab_data.csv'
#     data = pd.read_csv(filepath)

    def __init__(self, data):
        self.data = data




    def export_data(filepath):
        data = pd.read_csv(filepath)


        return data


    data = export_data('C:/Users/Dell/Projects/mind_set/ab_testing/ab_data.csv')







class N_required:

    def __init__(self,  required_n_1, required_n):
        self._required_n_1 = required_n_1
        self.required_n = required_n



    def required_n_1(effect_size_1, effect_size_2, power, alpha, ratio):
        required_n = sms.NormalIndPower().solve_power(
                effect_size=sms.proportion_effectsize(0.121, 0.123),
                power=0.85,
                alpha=0.01,
                ratio=1
            )  # Calculating sample size needed


        required_n = ceil(required_n) # Округляем в сторону большего значения при необходимости


        return required_n




    # Параметры можно настроить как нужно для задачи

    required_n = required_n_1(0.121, 0.123, 0.85, 0.01, 1)






class Preprocessing_Data(Export_Data):


    def __init__(self, data, data_crosstab, data_for_ab_test, preproc = None,\
                 conversion_rates_1 = None, preparation_for_Ztest = None):

#         self._export_data = export_data # добавить в инит export data при необходимости
        self.data = data
        self.data_crosstab = data_crosstab
        self.data_for_ab_test = data_for_ab_test
        self._preproc = preproc
        self._conversion_rates_1 = conversion_rates_1
        self._preparation_for_Ztest =  preparation_for_Ztest


    def preproc(data):

        # создаем перекрестную таблицу для того, чтобы удостовериться, что в контрольной группе
        # есть люди с старой и новой страницей

        data_crosstab = pd.crosstab(data['group'], data['landing_page'])

        # Делим данные на контрольную и тестовую группы
        #
        required_n = 100000
        # Меняем random_state для нескольких проверок p-value
        control_sample = data[data['group'] == 'control'].sample(n=required_n, random_state=22)
        control_1_sample = data[data['group'] == 'control'].sample(n=required_n - 80000, random_state=18)
        treatment_sample = data[data['group'] == 'treatment'].sample(n=required_n, random_state=22)

        data_for_ab_test = pd.concat([control_sample, control_1_sample, treatment_sample], axis=0)
        data_for_ab_test.reset_index(drop=True, inplace=True)


        # Делаем список из возврааемых переменных
        return [data_crosstab, data_for_ab_test]





    def conversion_rates_1(data):
#         conversion_rates = data.groupby('group')['converted']

#         std_p = lambda x: np.std(x, ddof=0)              # Std. deviation of the proportion
#         se_p = lambda x: stats.sem(x, ddof=0)            # Std. error of the proportion (std / sqrt(n))

#         conversion_rates = conversion_rates.agg([np.mean, std_p, se_p])
#         conversion_rates.columns = ['conversion_rate', 'std_deviation', 'std_error']


#         return conversion_rates.style.format('{:.3f}')


        se_p = lambda x: stats.sem(x, ddof=0)
        conversion_rates = data.groupby('group').agg({'converted': [np.mean, np.std, se_p]})
        conversion_rates.columns = ['np_mean_conversion_rate', 'std_deviation', 'std_error']
        return conversion_rates





class Conducting_A_A_tests:

    def __init__(self, data, conducting_Ztest_AA = None):
        self.data = data
        self._conducting_Ztest_AA = conducting_Ztest_AA



    def conducting_Ztest_AA(data):


        control_results = data[data['group'] == 'control']['converted']
        control_1_results = data[data['group'] == 'control']['converted']

        n_con = control_results.count()
        n_con_1 = control_1_results.count()
        successes = [control_results.sum(), control_1_results.sum()]
        nobs = [n_con, n_con_1]

        z_stat, pval = proportions_ztest(successes, nobs=nobs)
        (lower_con, lower_con_1), (upper_con, upper_con_1) = proportion_confint(successes, nobs=nobs, alpha=0.05)

        print("""

        Для АА-теста


        """)
        print(f'z statistic: {z_stat:.2f}')
        print(f'p-value: {pval:.3f}')
        print(f'ci 95% for control group: [{lower_con:.3f}, {upper_con:.3f}]')
        print(f'ci 95% for control_1 group: [{lower_con_1:.3f}, {upper_con_1:.3f}]')








class Conducting_A_B_tests:


    def __init__(self, data, preparation_for_Ztest_AB = None,conducting_Ztest_AB = None):
        self.data = data
        self._conducting_Ztest_AB = conducting_Ztest_AB



    def conducting_Ztest_AB(data):
        control_results = data[data['group'] == 'control']['converted']
        treatment_results = data[data['group'] == 'treatment']['converted']

        n_con = control_results.count()
        n_treat = treatment_results.count()
        successes = [control_results.sum(), treatment_results.sum()]
        nobs = [n_con, n_treat]

        z_stat, pval = proportions_ztest(successes, nobs=nobs)
        (lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes, nobs=nobs, alpha=0.05)

        print("""

        Для АБ-теста

        """)
        print(f'z statistic: {z_stat:.2f}')
        print(f'p-value: {pval:.3f}')
        print(f'ci 95% for control group: [{lower_con:.3f}, {upper_con:.3f}]')
        print(f'ci 95% for treatment group: [{lower_treat:.3f}, {upper_treat:.3f}]')




class Modeling_AA_AB_tests():



    def __init__(self):
        pass


    def plot_probab_contr_and_treat(size = 100):


        # for control
        np.random.seed()
        n=1
        p_contr=Preprocessing_Data.conversion_rates_1(Export_Data.data).iloc[0,0]

        # for treatment
        np.random.seed()
        n=1
        p_treat=Preprocessing_Data.conversion_rates_1(Export_Data.data).iloc[1,0]

        # for control_1
        np.random.seed()
        n=1
        p_contr_1=Preprocessing_Data.conversion_rates_1(Export_Data.data).iloc[0,0]



    # let us repeat our experiment for 100000 times

        x=np.random.binomial(n=n, p=p_contr, size=size)
        probs_100= [sum(np.random.binomial(n,p_contr,size=size) == i)/size for i in range(n+1)]
        plt.xticks(range(n+1))
        plt.plot(list(range(n+1)), probs_100, color='blue', marker='o')
        plt.xlabel('Number of Heads',fontsize=14)
        plt.ylabel('Probability',fontsize=14)


        x_1=np.random.binomial(n=n, p=p_treat, size=size)
        probs_100= [sum(np.random.binomial(n,p_treat,size=size) == i)/size for i in range(n+1)]
        plt.xticks(range(n+1))
        plt.plot(list(range(n+1)), probs_100, color='red', marker='o')
        plt.xlabel('Number of Heads',fontsize=14)
        plt.ylabel('Probability',fontsize=14)



        x_2=np.random.binomial(n=n, p=p_contr_1, size=size)
        probs_100= [sum(np.random.binomial(n,p_contr_1,size=size) == i)/size for i in range(n+1)]
        plt.xticks(range(n+1))
        plt.plot(list(range(n+1)), probs_100, color='black', marker='o')
        plt.xlabel('Number of Heads',fontsize=14)
        plt.ylabel('Probability',fontsize=14)



        n_con = size
        n_treat = size
        successes_contr_test = [x.sum(), x_1.sum()]
        nobs_contr_test = [n_con, n_treat]

        z_stat_contr_test, pval_contr_test = proportions_ztest(successes_contr_test, nobs=nobs_contr_test)
        (lower_con_contr_test, lower_treat_contr_test),\
        (upper_con_contr_test, upper_treat_contr_test) = proportion_confint(successes_contr_test, nobs=nobs_contr_test, alpha=0.05)

        print("""



        Z-Test for Control-Treatment



        """)
        print(f'z statistic_contr_test: {z_stat_contr_test:.2f}')
        print(f'p-value_contr_test: {pval_contr_test:.3f}')
        print(f'ci 95% for control group_contr_test: [{lower_con_contr_test:.3f}, {upper_con_contr_test:.3f}]')
        print(f'ci 95% for treatment group_contr_test: [{lower_treat_contr_test:.3f}, {upper_treat_contr_test:.3f}]')


        n_con = size
        n_con_1 = size
        successes_contr_contr_1 = [x.sum(), x_2.sum()]
        nobs_contr_contr_1 = [n_con, n_con_1]

        z_stat_contr_contr_1, pval_contr_contr_1 = proportions_ztest(successes_contr_contr_1, nobs=nobs_contr_contr_1)
        (lower_con_contr_contr_1, lower_treat_contr_contr_1), \
        (upper_con_contr_contr_1, upper_treat_contr_contr_1) =  proportion_confint(successes_contr_contr_1, nobs=nobs_contr_contr_1, alpha=0.05)

        print("""



        Z-Test for Control - Control_1



        """)
        print(f'z statistic_contr_contr_1: {z_stat_contr_contr_1:.2f}')
        print(f'p-value_contr_contr_1: {pval_contr_contr_1:.3f}')
        print(f'ci 95% for control group_contr_contr_1: [{lower_con_contr_contr_1:.3f}, {upper_con_contr_contr_1:.3f}]')
        print(f'ci 95% for treatment group_contr_contr_1: [{lower_treat_contr_contr_1:.3f}, {upper_treat_contr_contr_1:.3f}]')










class Visulation_different_methods:



    def __init__(self):
        pass





    def plotting_beta_distribution(a, b):

        fig, ax = plt.subplots(1, 1)
        mean, var, skew, kurt = beta.stats(a, b, moments='mvsk')

        x = np.linspace(beta.ppf(0.01, a, b),
                        beta.ppf(0.99, a, b), 100)
        ax.plot(x, beta.pdf(x, a, b),
               'r-', lw=5, alpha=0.9, label='beta pdf')


        rv = beta(a, b)
        ax.plot(x, rv.pdf(x), 'k-', lw=2, label='frozen pdf')


        vals = beta.ppf([0.001, 0.5, 0.999], a, b)
        np.allclose([0.001, 0.5, 0.999], beta.cdf(vals, a, b))


        r = beta.rvs(a, b, size=1000)



        ax.hist(r, density=True, histtype='stepfilled', alpha=0.2)
        ax.legend(loc='best', frameon=False)
        print("""
        mean: {}
        var: {}
        skew: {}
        kurt: {}
        """.format(mean, var, skew, kurt))
        plt.show()


    def modeling_AA_AB_tests(success_rate, N, NN, beta, skew):


    #     SELECTED PARAMETRES:

#         success_rate = 0.12
        uplift = 0.01
#         N = 5000
#         NN = 2000

#         beta = 7
#         skew = 1.44




        # VK-TEAM CODE


        # PARAMS

        skew_params = []
        for s in np.linspace(0.1, 4, 20):
            skew_params.append({'success_rate': success_rate, 'uplift': uplift,\
                                'beta': beta, 'skew': s, 'N': N, 'NN': NN})


        beta_params = []
        for b in np.logspace(0, 1, 20)[::-1]:
            beta_params.append({'success_rate': success_rate, 'uplift': uplift, \
                                'beta': b, 'skew': skew, 'N': N, 'NN': NN})

        sr_params = []
        for sr in np.logspace(-3, -0.3, 20):
            sr_params.append({'success_rate': sr, 'uplift': uplift, \
                              'beta': 500, 'skew': skew, 'N': N, 'NN': NN})


        # DATA-GENERATION
        # Параметр uplift регулируется до генерации данных
#         for param in tqdm(beta_params + skew_params + sr_params):
#             apply_all_tests('../data', **param)

        # PICS

        # Устанавливаем размер фигуры и её плотность на пиксель
        figsize = (4, 3)
        dpi = 200


        # с помощью коэффициента Манна-Уитни проверяем гипотезу:
        # Чем выше график ROC-AUC приближается к левой верхней части, тем лучше
        # Для простоты можно смотреть на мощность теста: она максимальная - 1

        #Вывод: Уверенно отвергается гипотеза о случайности разниц (!!!ДАННЫЕ СМЕНИТЬ)
        fig, ax = plt.subplots(figsize=figsize, dpi=dpi) # пример генерации данных a/b и a/a теста
        ab_data, aa_data = rpv('../data', 'mannwhitney_successes_count', N=N, NN=NN, uplift=uplift, beta=beta, success_rate=success_rate, skew=0.1)
        plot_cdf(ab_data, 'MW', ax)
        ax.plot(np.linspace(0, 1, 10000), np.linspace(0, 1, 10000), 'k', alpha=0.1)
        ax.axvline(0.05, color='k', alpha=0.5) # 0,05 показывает как проходит вертикальная серая линия
        ax.set_xlabel('FPR')
        ax.set_ylabel('Sensitivity')


        # Распределения просмотров у выборки А и В
        fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
        (views_0_ab, clicks_0_ab), _, gt_success_rates = generate_data(**skew_params[6])
        sns.distplot(views_0_ab.flatten(), bins=range(0, 40), ax=ax, kde=False, norm_hist=True)
        ax.set_title('Views distribution')




        # Распределение коэффиента конферсии на каждого пользователя
        fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
        sns.distplot(gt_success_rates.flatten(), ax=ax, kde=False, norm_hist=True)
        ax.set_title('Ground truth user CTR distribution')


        # Распределение кликов
        fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
        sns.distplot(clicks_0_ab.flatten(), bins=range(0, 10), ax=ax, kde=False, norm_hist=True)
        ax.set_title('Clicks distribution')


        # Просмотр результатов расчетов p-value и построения ROC-AUC кривой у АВ - ТЕСТА
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(figsize[0] * 2, figsize[1]), dpi=dpi)
        ab_data, aa_data = rpv('../data', 'mannwhitney_successes_count', N=N, NN=NN, uplift=uplift, beta=beta, success_rate=success_rate, skew=0.1)
        plot_cdf(ab_data, 'MW', ax2)

        ax2.plot(np.linspace(0, 1, 10000), np.linspace(0, 1, 10000), 'k', alpha=0.1)
        ax2.axvline(0.05, color='k', alpha=0.5)
        ax2.set_title('Simulated p-value CDFs under H1')
        ax2.set_xlabel('p-value')

        ax1.set_title('Simulated p-value histogram under H1')
        ax1.set_xlabel('p-value')
        sns.distplot(ab_data.flatten(), ax=ax1, bins=100, kde=False, norm_hist=True)


        # Встроенный АА - ТЕСТ и показ тех же самых результатов
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(figsize[0] * 2, figsize[1]), dpi=dpi)

        ab_data, aa_data = rpv('../data', 'mannwhitney_successes_count', N=N, NN=NN, uplift=uplift, beta=beta, success_rate=success_rate, skew=0.1)
        plot_cdf(aa_data, 'MW', ax2)

        ax2.plot(np.linspace(0, 1, 10000), np.linspace(0, 1, 10000), 'k', alpha=0.1)
        ax2.axvline(0.05, color='k', alpha=0.5)
        ax2.set_title('Simulated p-value CDFs under H0 (FPR)')
        ax2.set_xlabel('p-value')

        ax1.set_title('Simulated p-value histogram under H0')
        ax1.set_xlabel('p-value')
        sns.distplot(aa_data.flatten(), ax=ax1, bins=10, kde=False, norm_hist=True)


        # Интерпретация графиков
        # Левый верхний график - Проведение АБ теста
        # Правый верхний график - Проведение АА теста
        # Левый нижный график - Мощность теста (место пересечения ROC-AUC с вертикальной
        # линией в 0,05)
        # Нижние правые графики - показ распределения в зависимости от разницы количества
        # выборки и дисперсии

        frames = [frame_from_params('../data', p, codenames=['ttest_successes_count', 'mannwhitney_successes_count','bootstrap', 'ttest_smoothed', 'buckets_ctrs', 'weighted_bootstrap', 'binomial_test']) for p in skew_params]


        return save_gif_and_show('../gifs/preview.gif', frames)


# Visulation_different_methods.modeling_AA_AB_tests

    def vk_sandbox(success_rate, uplift, N, NN, beta, skew):

        # Задаем параметры модели
        ab_params = {'success_rate': success_rate, 'uplift': uplift, 'beta': beta, 'skew': skew, 'N': N, 'NN': NN}
        aa_params = {'success_rate': success_rate, 'uplift': 0.0, 'beta': beta, 'skew': skew, 'N': N, 'NN': NN}

        # Задаем графики распределений
        (views_0_ab, clicks_0_ab), (views_1_ab, clicks_1_ab), gt_success_rates = generate_data(**ab_params)
        (views_0_aa, clicks_0_aa), (views_1_aa, clicks_1_aa), _ = generate_data(**aa_params)

        # Задаем параметры модели
        linearized_0_ab, linearized_1_ab = linearization_of_clicks(clicks_0_ab, views_0_ab, clicks_1_ab, views_1_ab)
        linearized_0_aa, linearized_1_aa = linearization_of_clicks(clicks_0_aa, views_0_aa, clicks_1_aa, views_1_aa)

        corr_aware_w_0_ab, corr_aware_w_1_ab = intra_user_correlation_aware_weights(clicks_0_ab, views_0_ab, views_1_ab)
        corr_aware_w_0_aa, corr_aware_w_1_aa = intra_user_correlation_aware_weights(clicks_0_aa, views_0_aa, views_1_aa)

        smoothed_ctrs_0_ab, smoothed_ctrs_1_ab = get_smoothed_ctrs(clicks_0_ab, views_0_ab, clicks_1_ab, views_1_ab)
        smoothed_ctrs_0_aa, smoothed_ctrs_1_aa = get_smoothed_ctrs(clicks_0_aa, views_0_aa, clicks_1_aa, views_1_aa)

        global_ctr_0_ab = clicks_0_ab.sum(axis=1) / views_0_ab.sum(axis=1)
        global_ctr_1_ab = clicks_1_ab.sum(axis=1) / views_1_ab.sum(axis=1)
        global_ctr_0_aa = clicks_0_aa.sum(axis=1) / views_0_aa.sum(axis=1)
        global_ctr_1_aa = clicks_1_aa.sum(axis=1) / views_1_aa.sum(axis=1)

        results = {
            'ttest_successes_count': (t_test(clicks_0_ab, clicks_1_ab), t_test(clicks_0_aa, clicks_1_aa), colors[0]),
            'mannwhitney_successes_count': (mannwhitney(clicks_0_ab, clicks_1_ab), mannwhitney(clicks_0_aa, clicks_1_aa), colors[1]),
            'delta': (delta_method_ctrs(clicks_0_ab, views_0_ab, clicks_1_ab, views_1_ab),
                      delta_method_ctrs(clicks_0_aa, views_0_aa, clicks_1_aa, views_1_aa), colors[2]),
            'bootstrap': (bootstrap(clicks_0_ab / views_0_ab, views_0_ab, clicks_1_ab / views_1_ab, views_1_ab),
                          bootstrap(clicks_0_aa / views_0_aa, views_0_aa, clicks_1_aa / views_1_aa, views_1_aa), colors[3]),
            'linearization': (t_test(linearized_0_ab, linearized_1_ab), t_test(linearized_0_aa, linearized_1_aa), colors[4]),
            'buckets_ctrs': (bucketization(clicks_0_ab / views_0_ab, views_0_ab, clicks_1_ab / views_1_ab, views_1_ab),
                             bucketization(clicks_0_aa / views_0_aa, views_0_aa, clicks_1_aa / views_1_aa, views_0_aa), colors[5]),
            't_test_ctrs': (t_test(clicks_0_ab / views_0_ab, clicks_1_ab / views_1_ab),
                            t_test(clicks_0_aa / views_0_aa, clicks_1_aa / views_1_aa), colors[6]),
            'weighted_bootstrap': (bootstrap(clicks_0_ab / views_0_ab, corr_aware_w_0_ab, clicks_1_ab / views_1_ab, corr_aware_w_1_ab),
                                   bootstrap(clicks_0_aa / views_0_aa, corr_aware_w_0_aa, clicks_1_aa / views_1_aa, corr_aware_w_1_aa), colors[7]),
            'ttest_smoothed': (t_test(smoothed_ctrs_0_ab, smoothed_ctrs_1_ab), t_test(smoothed_ctrs_0_aa, smoothed_ctrs_1_aa), colors[8]),
            'binomial_test': (binomial_test(global_ctr_0_ab, views_0_ab.sum(axis=1), global_ctr_1_ab, views_1_ab.sum(axis=1)),
                          binomial_test(global_ctr_0_aa, views_0_aa.sum(axis=1), global_ctr_1_aa, views_1_aa.sum(axis=1)), colors[9])
    }


        # Создание списка для дальнейшего вырисовывания результатов
        pretty_results = {}
        for codename, p_values in results.items():
            pretty_title = codename
            if codename in codenames2titles:
                pretty_title = codenames2titles[codename][0]
            pretty_results[pretty_title] = p_values


        # вырисовка результатов
        plot_summary(pretty_results, views_0_aa, gt_success_rates)
        return plt.show()





def _main():


    # Экспорт данных
    #
    export = Export_Data.data
    print(export)






    # Необходимое количество юзеров для стат значимых результатов
    #
    n_required =  N_required.required_n
    print(f"""




    Для того, чтобы результаты были статистически значимы и интерпретируемы,
    нужно минимум {n_required} юзеров



    """)




    # вывдение перекрестной таблицы
    #
    preprocessing_data_data_crosstab = Preprocessing_Data.preproc(Export_Data.data)[0]
    print("""



    """, preprocessing_data_data_crosstab)

    # представление таблицы в виде двух выборок : контрольной и тестовой
    #
    preprocessing_data_data_for_ab_test = Preprocessing_Data.preproc(Export_Data.data)[1]
    print("""



     """, preprocessing_data_data_for_ab_test)




    # выведение таблицы со средним, стандартным отклонением
    #

    display(Preprocessing_Data.conversion_rates_1(Preprocessing_Data.preproc(Export_Data.data)[1]))



    # Проведение Z-теста для АА теста




    print(Conducting_A_A_tests.conducting_Ztest_AA(Preprocessing_Data.preproc(Export_Data.data)[1]))






    # Проведение Z-теста для AB теста

    print(Conducting_A_B_tests.conducting_Ztest_AB(Preprocessing_Data.preproc(Export_Data.data)[1]))




    # Моделирование данных




    plot_probab_contr_and_treat = Modeling_AA_AB_tests.plot_probab_contr_and_treat
    print("""



Моделирование данных с помощью биномиального распределения в зависимости от размера выборки:

При размерах выборки больше 800 тысяч p-value уверенно меньше порогового значения a = 0,05
При таких размерах выборки доверительный интервал выборочного среднего Контрольной выборки сдвинут
правее чем интервал у Тестовой выборки. Вывод: при размерах выборки 800 тысяч уверенно отвергается
гипотеза о случайности разниц средних: конверсия на контрольной выборки лучше чем на тестовой

При меньших размерах выборки p-value неустойчивое и часто больше порогового значения a = 0,05


Сравнение контрольных групп (разница будет из-за разницы, с которой работает функция random):
На малых выборках(до 10 тысяч наблюдений) разница может быть значительной
После нескольких сотен тысяч сгенерированных данных доверительный интервал сокращается и
p-value показывает значение больше порогового значения a = 0,05,что говорит
о случайной разнице между средними распределений



    """)
    size_slider = IntSlider(min=100, max=1000000, step=100, value=100, description='$\\nu$')
    interact(plot_probab_contr_and_treat,size=size_slider)



    # Дальнейшая визуализация  с классом Visulation_different_methods

    # Я хотел бы это показать на ответе доклада в зуме, так как это будет долго грузится

    # Функция подбора параметров
    plotting_beta_distribution = Visulation_different_methods.plotting_beta_distribution
    plotting_beta_distribution(1, 7)


    # Функция для визуализации gif
    modeling_AA_AB_tests = Visulation_different_methods.modeling_AA_AB_tests
    display(modeling_AA_AB_tests(0.12, 5000, 2000, 7, 1.44))
    print("""



Судя по графикам видим:
При конверсии на 1 процент ни один тест не показывает статистическую значимость.
При увеличении количества выборки на АА тесте Z-тест дает большой FPR
и видит разницу между двумя приблизительно одинаковыми выборками
Вывод: при такой маленькой конверсии и количеством данных до миллиона нужно
сделать вывод: разница между двумя выборками случайна




    """)


    # Функция sandbox для для визуализации результатов всех тесто
    vk_sandbox = Visulation_different_methods.vk_sandbox
    vk_sandbox(0.12, 0.01, 5000, 2000, 7, 1.44)

    print("""



Выводы по sandbox:
При проведении АА теста только Z-test дает высокий FPR, остальные графики однородны
При АВ тесте все графики показывают низкий FPR, кроме z-теста:
при большой ассиметрии данных он дает сбои и отвергает нулевую гипотезу уверенно
несмотря на результаты остальных тестов
Вывод: при конверсии на 1% и выборке меньше 1 миллиона, гипотеза о статистической
незначимости разницы принимается



    """)




if __name__ == '__main__':
    _main()



